In [1]:
# %load_ext rpy2.ipython 
# import vamb
%config IPCompleter.greedy=False
%config Completer.use_jedi=False
# %load_ext rpy2.ipython
%matplotlib inline


In [3]:
import pickle
import numpy as np
import pandas as pd
import os
import glob
import requests
import webbrowser
# import xmltodict
import json
from itertools import compress
import re
import seaborn as sns
import plotly as pl
import matplotlib
import matplotlib.pyplot as plt
from Bio import SeqIO
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew
import plotly.io as pio
import plotly.express as px
pio.renderers.default = 'iframe_connected'
plt.rcParams['figure.figsize'] = [10, 10]

Functions

In [4]:
from IPython.display import HTML

def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))

In [30]:
%cd /home/gladkov2/storage/metagenome/rnf_c_new/
!ls -halt

[Errno 2] No such file or directory: '/home/gladkov2/storage/metagenome/rnf_c_new/'
/mnt/storage/soil_institute/graphmb
total 14G
drwxrwxrwx 15 zverevgpu  zverevgpu  4.0K Aug  2 12:29 ..
drwxrwxr-x 13 gladkovgpu gladkovgpu 4.0K Aug  2 12:26 .
-rw-rw-r--  1 gladkovgpu gladkovgpu  70K Aug  2 12:26 docu-docu.ipynb
-rw-rw-r--  1 gladkovgpu gladkovgpu 4.8K Jul 31 16:28 clusters.ipynb
drwxrwxr-x  2 gladkovgpu gladkovgpu 4.0K Jul 31 16:20 .virtual_documents
drwxrwxr-x  2 gladkovgpu gladkovgpu 4.0K Jul 31 16:19 .ipynb_checkpoints
-rw-rw-r--  1 gladkovgpu gladkovgpu 2.3K Jul 31 16:19 Untitled.ipynb
drwxrwxr-x  2 gladkovgpu gladkovgpu 4.0K Jul 28 18:02 iframe_figures
drwxrwxr-x  8 gladkovgpu gladkovgpu 4.0K Jul  5 15:08 second_run
-rw-rw-r--  1 gladkovgpu gladkovgpu 5.2M Jul  3 15:15 checkm_edges_polished_results.txt
drwxrwxr-x  2 gladkovgpu gladkovgpu 4.0K Jun 29 17:50 results2
-rw-rw-r--  1 gladkovgpu gladkovgpu  854 Jun 29 17:31 scripts.bash
drwxrwxr-x  4 gladkovgpu gladkovgpu 4.0K Jun 29 17:

In [3]:
# %cd /home/gladkovgpu/soil_institute/assembled_by_repeates/
path = "/home/gladkovgpu/soil_institute/assembled_by_repeates/"

In [87]:
!ls 1/anno_new/

ls: cannot access '1/anno_new/': No such file or directory


In [4]:
files = sorted(glob.glob(f"{os.getcwd()}/*/anno/*.emapper.annotations"))
files

[]

In [6]:
files = sorted(glob.glob(f"{path}/*/anno/*.emapper.annotations"))
files

['/home/gladkovgpu/soil_institute/assembled_by_repeates/1/anno/1.emapper.annotations',
 '/home/gladkovgpu/soil_institute/assembled_by_repeates/2/anno/2.emapper.annotations',
 '/home/gladkovgpu/soil_institute/assembled_by_repeates/3/anno/3.emapper.annotations',
 '/home/gladkovgpu/soil_institute/assembled_by_repeates/4/anno/4.emapper.annotations',
 '/home/gladkovgpu/soil_institute/assembled_by_repeates/5/anno/5.emapper.annotations']

In [7]:
# names = [os.path.basename(i).split(".")[0] for i in files]
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))
header = pd.read_csv(file_list[1][1],sep="\t", skiprows=4 ,nrows=0).columns.to_list()
dfs = [pd.read_csv(f[1], sep="\t", comment="#", header=None, names=header).assign(site=f[0]) for f in file_list]
df = pd.concat(dfs, ignore_index=True)

In [9]:
df

,#query,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs,site
0,contig_1_1,575540.Isop_2634,5.820000e-144,466.0,"COG1850@1|root,COG1850@2|Bacteria",2|Bacteria,G,ribulose-bisphosphate carboxylase activity,cbbL,-,...,"ko00630,ko00710,ko01100,ko01120,ko01200,map006...","M00165,M00166,M00532","R00024,R03140","RC00172,RC00859","ko00000,ko00001,ko00002,ko01000",-,-,-,"RuBisCO_large,RuBisCO_large_N",1
1,contig_1_2,1089551.KE386572_gene2869,1.080000e-128,417.0,"COG0673@1|root,COG0673@2|Bacteria,1MWNK@1224|P...",28211|Alphaproteobacteria,S,"Oxidoreductase family, NAD-binding Rossmann fold",-,-,...,-,-,-,-,-,-,-,-,"GFO_IDH_MocA,GFO_IDH_MocA_C",1
2,contig_1_3,886293.Sinac_2235,7.650000e-55,202.0,"COG2267@1|root,COG2267@2|Bacteria",2|Bacteria,I,carboxylic ester hydrolase activity,-,-,...,-,-,-,-,-,-,-,-,"Abhydrolase_1,Abhydrolase_6,Hydrolase_4",1
3,contig_1_4,536019.Mesop_5823,7.420000e-28,122.0,"COG0036@1|root,COG0036@2|Bacteria,1RF86@1224|P...",28211|Alphaproteobacteria,G,Ribulose-phosphate 3 epimerase family,-,-,...,"ko00030,ko00040,ko00710,ko01100,ko01110,ko0112...","M00004,M00007",R01529,RC00540,"ko00000,ko00001,ko00002,ko01000",-,-,-,Ribul_P_3_epim,1
4,contig_1_0,886293.Sinac_2234,7.650000e-264,810.0,"COG1387@1|root,COG1796@1|root,COG1387@2|Bacter...",203682|Planctomycetes,L,DNA polymerase,-,-,...,-,-,-,-,"ko00000,ko03400",-,-,-,"DNA_pol_B_palm,DNA_pol_B_thumb,HHH_5,HHH_8,PHP",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212486,contig_9999_19,296591.Bpro_3466,8.500000e-13,82.4,"COG0620@1|root,COG0620@2|Bacteria,1MUI9@1224|P...",28216|Betaproteobacteria,H,Catalyzes the transfer of a methyl group from ...,metE,-,...,"ko00270,ko00450,ko01100,ko01110,ko01230,map002...",M00017,"R04405,R09365","RC00035,RC00113,RC01241","ko00000,ko00001,ko00002,ko01000",-,-,-,"Meth_synt_1,Meth_synt_2",5
2212487,contig_9999_13,509190.Cseg_0476,1.850000e-59,226.0,"COG2272@1|root,COG2272@2|Bacteria,1MVQZ@1224|P...",28211|Alphaproteobacteria,I,Belongs to the type-B carboxylesterase lipase ...,-,-,...,-,-,-,-,"ko00000,ko01000",-,CE10,-,COesterase,5
2212488,contig_9999_5,935863.AWZR01000004_gene416,7.390000e-118,393.0,"COG1858@1|root,COG1858@2|Bacteria,1REQ9@1224|P...",1224|Proteobacteria,P,cytochrome C peroxidase,-,-,...,-,-,-,-,-,-,-,-,CCP_MauG,5
2212489,contig_9999_2,1122918.KB907249_gene3906,4.390000e-146,470.0,"COG2124@1|root,COG2124@2|Bacteria,1TPWZ@1239|F...",91061|Bacilli,Q,cytochrome P450,-,-,...,-,-,-,-,-,-,-,-,p450,5


In [9]:
pd.set_option('display.max_columns', None)
df.head()

,#query,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,EC,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs,site
0,contig_1_1,575540.Isop_2634,5.820000e-144,466.0,"COG1850@1|root,COG1850@2|Bacteria",2|Bacteria,G,ribulose-bisphosphate carboxylase activity,cbbL,-,4.1.1.39,ko:K01601,"ko00630,ko00710,ko01100,ko01120,ko01200,map006...","M00165,M00166,M00532","R00024,R03140","RC00172,RC00859","ko00000,ko00001,ko00002,ko01000",-,-,-,"RuBisCO_large,RuBisCO_large_N",1
1,contig_1_2,1089551.KE386572_gene2869,1.080000e-128,417.0,"COG0673@1|root,COG0673@2|Bacteria,1MWNK@1224|P...",28211|Alphaproteobacteria,S,"Oxidoreductase family, NAD-binding Rossmann fold",-,-,-,-,-,-,-,-,-,-,-,-,"GFO_IDH_MocA,GFO_IDH_MocA_C",1
2,contig_1_3,886293.Sinac_2235,7.650000e-55,202.0,"COG2267@1|root,COG2267@2|Bacteria",2|Bacteria,I,carboxylic ester hydrolase activity,-,-,-,-,-,-,-,-,-,-,-,-,"Abhydrolase_1,Abhydrolase_6,Hydrolase_4",1
3,contig_1_4,536019.Mesop_5823,7.420000e-28,122.0,"COG0036@1|root,COG0036@2|Bacteria,1RF86@1224|P...",28211|Alphaproteobacteria,G,Ribulose-phosphate 3 epimerase family,-,-,5.1.3.1,ko:K01783,"ko00030,ko00040,ko00710,ko01100,ko01110,ko0112...","M00004,M00007",R01529,RC00540,"ko00000,ko00001,ko00002,ko01000",-,-,-,Ribul_P_3_epim,1
4,contig_1_0,886293.Sinac_2234,7.650000e-264,810.0,"COG1387@1|root,COG1796@1|root,COG1387@2|Bacter...",203682|Planctomycetes,L,DNA polymerase,-,-,-,ko:K02347,-,-,-,-,"ko00000,ko03400",-,-,-,"DNA_pol_B_palm,DNA_pol_B_thumb,HHH_5,HHH_8,PHP",1


In [10]:
df = df.replace('-', np.nan)

In [11]:
(df.count() / df.count()[0] * 100).round()

#query            100.0
seed_ortholog     100.0
evalue            100.0
score             100.0
eggNOG_OGs        100.0
max_annot_lvl     100.0
COG_category       97.0
Description        97.0
Preferred_name     33.0
GOs                 8.0
EC                 31.0
KEGG_ko            55.0
KEGG_Pathway       35.0
KEGG_Module        24.0
KEGG_Reaction      25.0
KEGG_rclass        24.0
BRITE              55.0
KEGG_TC            11.0
CAZy                1.0
BiGG_Reaction       1.0
PFAMs              95.0
site              100.0
dtype: float64

In [12]:
df.shape[0]

2212491

In [13]:
df.groupby("KEGG_ko").count()["#query"].sort_values(ascending=False)

KEGG_ko
ko:K03088                        10421
ko:K01989                         9050
ko:K01990                         6273
ko:K00059                         5216
ko:K02035                         4695
                                 ...  
ko:K08483,ko:K11189                  1
ko:K01139,ko:K03790                  1
ko:K02147                            1
ko:K02160,ko:K15037,ko:K18605        1
ko:K05365,ko:K21464                  1
Name: #query, Length: 13313, dtype: int64

In [14]:
df.groupby(["KEGG_ko", "site"]).count()["#query"].sort_values(ascending=False)

KEGG_ko              site
ko:K01989            2       3899
ko:K03088            4       2836
                     1       2232
                     2       2135
ko:K01990            4       1834
                             ... 
ko:K06122            5          1
ko:K06121,ko:K13919  2          1
ko:K06119            1          1
ko:K01854,ko:K07011  2          1
ko:K22502            3          1
Name: #query, Length: 43522, dtype: int64

In [29]:
!python /home/gladkovgpu/app/MinPath/MinPath.py

minpath directory: /home/gladkovgpu/app/MinPath
Usage: python MinPath.py <-ko filename>/<-ec filename>/<-fig filename>/<-any annfile> [-map mapfile] [-report filename] [-details detailed-output]
Note: your input file can contain functional annotations in either of the following
   -ko file: annotation in KEGG KO families
   -ec file: annotation in EC numbers (to be mapped to MetaCyc pathways)
   -fig file: annotation in SEED fig families
   -any file: annotation in any families, then you must specify -map, the pathway-function mapping file
Example 1: python MinPath.py -ko demo.ko -report demo.ko.minpath
Example 2: python MinPath.py -ko demo.ko -report demo.ko.minpath -details demo.ko.minpath.details
Example 3: python MinPath.py -ec demo.ec -report demo.ec.minpath -details demo.ec.minpath.details
Example 4: python MinPath.py -fig demo.fig -report demo.fig.minpath -details demo.fig.minpath.details
Example 5: python MinPath.py -any demo.ec -map data/ec2path -report demo.any.minpath -detai

In [32]:
!cat /home/gladkovgpu/app/MinPath/examples/demo.ko | head

34765050	K00844
34765059	K02256
34765098	K02274
34765113	K03878
34765115	K03880
34765153	K07005
34765176	K00826
34765194	K01256
34765198	K00342
34765244	K10896
cat: write error: Broken pipe


In [ ]:
df.loc()

In [15]:
%cd ..
%mkdir postprocessing
%cd postprocessing

/mnt/storage/soil_institute
/mnt/storage/soil_institute/postprocessing


In [16]:
!ls -ahlt --color

total 8.0K
drwxrwxr-x  2 gladkovgpu gladkovgpu 4.0K Jul 28 14:08 .
drwxrwxrwx 14 zverevgpu  zverevgpu  4.0K Jul 28 14:08 ..


In [17]:
for i in df["site"].unique().tolist():
    df.loc[df["site"] == i]["KEGG_ko"].dropna().str.split(",").explode().str[3:].to_csv(f"{i}_keggs", sep="\t")

In [62]:
!head keggs_all

	KEGG_ko
0	K01601
3	K01783
4	K02347
5	K01607
6	K07302
9	K07481
17	K07722
18	K07486
21	K05820


In [65]:
!head result_minpath_all

path 00010 kegg n/a  naive 1  minpath 1  fam0  105  fam-found  89  name  Glycolysis / Gluconeogenesis
path 00020 kegg n/a  naive 1  minpath 1  fam0  62  fam-found  50  name  Citrate cycle (TCA cycle)
path 00030 kegg n/a  naive 1  minpath 1  fam0  84  fam-found  66  name  Pentose phosphate pathway
path 00040 kegg n/a  naive 1  minpath 1  fam0  85  fam-found  65  name  Pentose and glucuronate interconversions
path 00051 kegg n/a  naive 1  minpath 1  fam0  111  fam-found  92  name  Fructose and mannose metabolism
path 00052 kegg n/a  naive 1  minpath 1  fam0  77  fam-found  45  name  Galactose metabolism
path 00053 kegg n/a  naive 1  minpath 1  fam0  61  fam-found  43  name  Ascorbate and aldarate metabolism
path 00061 kegg n/a  naive 1  minpath 1  fam0  39  fam-found  33  name  Fatty acid biosynthesis
path 00062 kegg n/a  naive 1  minpath 0  fam0  24  fam-found  8  name  Fatty acid elongation
path 00071 kegg n/a  naive 1  minpath 1  fam0  58  fam-found  43  name  Fatty acid degradation


In [94]:
!python /home/gladkovgpu/app/MinPath/MinPath.py --version

minpath directory: /home/gladkovgpu/app/MinPath
Usage: python MinPath.py <-ko filename>/<-ec filename>/<-fig filename>/<-any annfile> [-map mapfile] [-report filename] [-details detailed-output]
Note: your input file can contain functional annotations in either of the following
   -ko file: annotation in KEGG KO families
   -ec file: annotation in EC numbers (to be mapped to MetaCyc pathways)
   -fig file: annotation in SEED fig families
   -any file: annotation in any families, then you must specify -map, the pathway-function mapping file
Example 1: python MinPath.py -ko demo.ko -report demo.ko.minpath
Example 2: python MinPath.py -ko demo.ko -report demo.ko.minpath -details demo.ko.minpath.details
Example 3: python MinPath.py -ec demo.ec -report demo.ec.minpath -details demo.ec.minpath.details
Example 4: python MinPath.py -fig demo.fig -report demo.fig.minpath -details demo.fig.minpath.details
Example 5: python MinPath.py -any demo.ec -map data/ec2path -report demo.any.minpath -detai

### minpath

In [41]:
%cd ../postprocessing

/mnt/storage/soil_institute/postprocessing


In [ ]:
for i in df["site"].unique().tolist():
    !python /home/gladkovgpu/app/MinPath/MinPath.py -ko {i}_keggs -report result_minpath_site_{i} -details detailed_output_minpath_site_{i}

In [109]:
minpath = pd.read_csv("result_minpath_site_1",sep="  ",engine='python', names=["hui0", "hui", "hui2", "minpath", "hui3", "fam0", "fam_found", "hui5", "name"])
minpath

,hui0,hui,hui2,minpath,hui3,fam0,fam_found,hui5,name
0,path 00010 kegg n/a,naive 1,minpath 1,fam0,105,fam-found,76,name,Glycolysis / Gluconeogenesis
1,path 00020 kegg n/a,naive 1,minpath 1,fam0,62,fam-found,47,name,Citrate cycle (TCA cycle)
2,path 00030 kegg n/a,naive 1,minpath 1,fam0,84,fam-found,60,name,Pentose phosphate pathway
3,path 00040 kegg n/a,naive 1,minpath 1,fam0,85,fam-found,57,name,Pentose and glucuronate interconversions
4,path 00051 kegg n/a,naive 1,minpath 1,fam0,111,fam-found,77,name,Fructose and mannose metabolism
...,...,...,...,...,...,...,...,...,...
417,path 05414 kegg n/a,naive 1,minpath 0,fam0,87,fam-found,2,name,Dilated cardiomyopathy
418,path 05415 kegg n/a,naive 1,minpath 0,fam0,160,fam-found,27,name,Diabetic cardiomyopathy
419,path 05416 kegg n/a,naive 1,minpath 0,fam0,39,fam-found,1,name,Viral myocarditis
420,path 05417 kegg n/a,naive 1,minpath 0,fam0,159,fam-found,10,name,Lipid and atherosclerosis


In [4]:
!pwd

/mnt/storage/soil_institute/graphmb


In [13]:
from pathlib import Path

path = r'/mnt/storage/soil_institute/postprocessing'
files = Path(path).glob('result_minpath_site_*')

dfs = list()
for f in files:
    data = pd.read_csv(f,sep="  ",engine='python', names=["hui0", "hui1", "hui2", "hui22", "fam0", "hui3", "fam_found", "hui5", "name"])
    data['site'] = f.stem
    dfs.append(data)

minpath = pd.concat(dfs, ignore_index=True)
minpath = minpath.drop(list(minpath.filter(regex='hui')), axis=1)
cols = minpath.columns.tolist()
cols = cols[-1:] + cols[:-1]
minpath = minpath[cols]
minpath["site"] = minpath["site"].str.split("_").str[3]
minpath = minpath.sort_values("site")
minpath


,site,fam0,fam_found,name
2094,1,99,7,Fluid shear stress and atherosclerosis
1806,1,109,72,Sulfur metabolism
1807,1,22,17,Caprolactam degradation
1808,1,38,8,Phenylpropanoid biosynthesis
1809,1,27,1,Flavonoid biosynthesis
...,...,...,...,...
1138,5,53,5,Thyroid hormone synthesis
1139,5,91,6,Thyroid hormone signaling pathway
1140,5,53,8,Adipocytokine signaling pathway
1168,5,287,34,Alzheimer disease


In [43]:
View(min_sorted)

NameError: name 'View' is not defined

In [14]:
minpath["rel"] = minpath["fam_found"].div(minpath.groupby('site')["fam_found"].transform('sum'))
minpath["rel"] = minpath["rel"].transform(lambda x: round(x*100, 2))
min_sorted = minpath.sort_values("fam_found", ascending=False).groupby("site").head(3000)
min_sorted

,site,fam0,fam_found,name,rel
1411,2,512,358,ABC transporters,4.80
570,4,512,339,ABC transporters,4.96
988,5,512,326,ABC transporters,4.82
163,3,512,316,ABC transporters,5.06
1412,2,495,311,Two-component system,4.17
...,...,...,...,...,...
686,4,75,1,Leukocyte transendothelial migration,0.01
684,4,64,1,Fc gamma R-mediated phagocytosis,0.01
683,4,86,1,Th17 cell differentiation,0.01
682,4,67,1,Th1 and Th2 cell differentiation,0.01


In [15]:
import plotly.express as px


fig = px.bar(min_sorted, x="site", y="rel", color="name", title="Minpath output")
fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['1','2','3','4', '5']})
fig.update_layout(autosize=False,width=1200,height=800)
fig.show()

In [25]:
%micromamba env list

UsageError: Line magic function `%micromamba` not found.


In [290]:
dff = df.copy()
dff = dff[["#query", "KEGG_ko", "site"]]
dff['id'] = dff['site'] + "_" + dff['#query'].str.split("_").str[:2].str.join("_")  
# dff['#query'].str.split("_")[:-2].str.join("_") 
dff
# dff['#query'].str.split("_").str[:2].str.join("_") 

,#query,KEGG_ko,site,id
0,contig_1_1,ko:K01601,1,1_contig_1
1,contig_1_2,NaN,1,1_contig_1
2,contig_1_3,NaN,1,1_contig_1
3,contig_1_4,ko:K01783,1,1_contig_1
4,contig_1_0,ko:K02347,1,1_contig_1
...,...,...,...,...
2212486,contig_9999_19,ko:K00549,5,5_contig_9999
2212487,contig_9999_13,ko:K03929,5,5_contig_9999
2212488,contig_9999_5,NaN,5,5_contig_9999
2212489,contig_9999_2,NaN,5,5_contig_9999


In [291]:
dfpaf['id'] = dff['site'].str.split(".").str[0]  + "_" + dfpaf['contigName']
dfpaf

,contigName,totalAvgDepth,site,id
0,contig_56336,0.000000,1.1,1_contig_56336
1,contig_29215,0.166206,1.1,1_contig_29215
2,contig_8522,0.121339,1.1,1_contig_8522
3,contig_58210,0.000000,1.1,1_contig_58210
4,contig_17370,0.000000,1.1,1_contig_17370
...,...,...,...,...
206955,contig_19175,0.992574,2.2,1_contig_19175
206956,contig_47766,0.000000,2.2,1_contig_47766
206957,contig_7603,0.000000,2.2,1_contig_7603
206958,contig_24071,0.000000,2.2,1_contig_24071


In [292]:
dff = dff.merge(dfpaf, on='id', how='left')
dff

,#query,KEGG_ko,site_x,id,contigName,totalAvgDepth,site_y
0,contig_1_1,ko:K01601,1,1_contig_1,contig_1,0.023852,1.1
1,contig_1_1,ko:K01601,1,1_contig_1,contig_1,0.000000,1.2
2,contig_1_1,ko:K01601,1,1_contig_1,contig_1,0.000000,1.3
3,contig_1_1,ko:K01601,1,1_contig_1,contig_1,0.000000,2.1
4,contig_1_1,ko:K01601,1,1_contig_1,contig_1,0.000000,2.2
...,...,...,...,...,...,...,...
3744208,contig_9999_19,ko:K00549,5,5_contig_9999,NaN,NaN,NaN
3744209,contig_9999_13,ko:K03929,5,5_contig_9999,NaN,NaN,NaN
3744210,contig_9999_5,NaN,5,5_contig_9999,NaN,NaN,NaN
3744211,contig_9999_2,NaN,5,5_contig_9999,NaN,NaN,NaN


In [293]:
dff = dff.loc[dff["site_y"] == "1.1"].dropna(subset="KEGG_ko")
dff["KEGG_ko"] = dff["KEGG_ko"].str.split(",")
#dff = dff.explode("KEGG_ko")
dff

,#query,KEGG_ko,site_x,id,contigName,totalAvgDepth,site_y
0,contig_1_1,[ko:K01601],1,1_contig_1,contig_1,0.023852,1.1
15,contig_1_4,[ko:K01783],1,1_contig_1,contig_1,0.023852,1.1
20,contig_1_0,[ko:K02347],1,1_contig_1,contig_1,0.023852,1.1
25,contig_10001_0,[ko:K01607],1,1_contig_10001,contig_10001,0.000000,1.1
28,contig_10004_7,[ko:K07302],1,1_contig_10004,contig_10004,0.005671,1.1
...,...,...,...,...,...,...,...
2002749,contig_9999_13,[ko:K00116],1,1_contig_9999,contig_9999,0.322653,1.1
2002758,contig_9999_6,[ko:K07755],1,1_contig_9999,contig_9999,0.322653,1.1
2002761,contig_9999_9,[ko:K03741],1,1_contig_9999,contig_9999,0.322653,1.1
2002767,contig_9999_2,[ko:K00384],1,1_contig_9999,contig_9999,0.322653,1.1


In [299]:
dff = dff.explode("KEGG_ko")
dff["KEGG_ko"] = dff["KEGG_ko"].str[3:]
dff

,#query,KEGG_ko,site_x,id,contigName,totalAvgDepth,site_y
0,contig_1_1,K01601,1,1_contig_1,contig_1,0.023852,1.1
15,contig_1_4,K01783,1,1_contig_1,contig_1,0.023852,1.1
20,contig_1_0,K02347,1,1_contig_1,contig_1,0.023852,1.1
25,contig_10001_0,K01607,1,1_contig_10001,contig_10001,0.000000,1.1
28,contig_10004_7,K07302,1,1_contig_10004,contig_10004,0.005671,1.1
...,...,...,...,...,...,...,...
2002749,contig_9999_13,K00116,1,1_contig_9999,contig_9999,0.322653,1.1
2002758,contig_9999_6,K07755,1,1_contig_9999,contig_9999,0.322653,1.1
2002761,contig_9999_9,K03741,1,1_contig_9999,contig_9999,0.322653,1.1
2002767,contig_9999_2,K00384,1,1_contig_9999,contig_9999,0.322653,1.1


In [340]:
dff.loc[(dff["KEGG_ko"]=="K22503") & (dff["site_y"]=="1.1")]

,#query,KEGG_ko,site_x,id,contigName,totalAvgDepth,site_y,count,weight
463858,contig_20711_0,K22503,1,1_contig_20711,contig_20711,0.266392,1.1,5,1.331960
508570,contig_21643_1,K22503,1,1_contig_21643,contig_21643,0.291167,1.1,5,1.455835
672988,contig_26079_2,K22503,1,1_contig_26079,contig_26079,0.342684,1.1,5,1.713420
711048,contig_27091_0,K22503,1,1_contig_27091,contig_27091,0.000000,1.1,5,0.000000
1412395,contig_48156_1,K22503,1,1_contig_48156,contig_48156,0.007986,1.1,5,0.039928


In [325]:
dff["count"] = dff[["id","KEGG_ko","totalAvgDepth", "site_y", "site_x"]].groupby(["site_y","KEGG_ko"]).transform('count')["id"]

In [333]:
dff[["id","KEGG_ko","totalAvgDepth", "site_y", "site_x", "count"]].drop_duplicates(ignore_index=True)

,id,KEGG_ko,totalAvgDepth,site_y,site_x,count
0,1_contig_1,K01601,0.023852,1.1,1,9
1,1_contig_1,K01783,0.023852,1.1,1,163
2,1_contig_1,K02347,0.023852,1.1,1,71
3,1_contig_10001,K01607,0.000000,1.1,1,208
4,1_contig_10004,K07302,0.005671,1.1,1,318
...,...,...,...,...,...,...
270353,1_contig_9999,K00116,0.322653,1.1,1,19
270354,1_contig_9999,K07755,0.322653,1.1,1,119
270355,1_contig_9999,K03741,0.322653,1.1,1,157
270356,1_contig_9999,K00384,0.322653,1.1,1,514


In [334]:
dff["weight"] = dff["count"]*dff["totalAvgDepth"]
dff

,#query,KEGG_ko,site_x,id,contigName,totalAvgDepth,site_y,count,weight
0,contig_1_1,K01601,1,1_contig_1,contig_1,0.023852,1.1,9,0.214664
15,contig_1_4,K01783,1,1_contig_1,contig_1,0.023852,1.1,163,3.887795
20,contig_1_0,K02347,1,1_contig_1,contig_1,0.023852,1.1,71,1.693457
25,contig_10001_0,K01607,1,1_contig_10001,contig_10001,0.000000,1.1,208,0.000000
28,contig_10004_7,K07302,1,1_contig_10004,contig_10004,0.005671,1.1,318,1.803260
...,...,...,...,...,...,...,...,...,...
2002749,contig_9999_13,K00116,1,1_contig_9999,contig_9999,0.322653,1.1,19,6.130407
2002758,contig_9999_6,K07755,1,1_contig_9999,contig_9999,0.322653,1.1,119,38.395707
2002761,contig_9999_9,K03741,1,1_contig_9999,contig_9999,0.322653,1.1,157,50.656521
2002767,contig_9999_2,K00384,1,1_contig_9999,contig_9999,0.322653,1.1,514,165.843642


In [339]:
dff[["KEGG_ko","site_y", "site_x", "weight"]].groupby(["KEGG_ko","site_y", "site_x"]).sum()

,,,weight
KEGG_ko,site_y,site_x,
K00001,1.1,1,19762.937854
K00002,1.1,1,0.159917
K00003,1.1,1,2452.031191
K00004,1.1,1,197.963908
K00005,1.1,1,0.192027
...,...,...,...
K22478,1.1,1,0.240951
K22479,1.1,1,93.354585
K22486,1.1,1,0.105933


In [312]:
dff[["id","KEGG_ko","totalAvgDepth", "site_y", "site_x"]].groupby(["site_y","KEGG_ko"]).count()

id  totalAvgDepth  site_x
site_y KEGG_ko                            
1.1    K00001   421            421     421
       K00002     2              2       2
       K00003   144            144     144
       K00004    49             49      49
       K00005     4              4       4
...             ...            ...     ...
       K22478     1              1       1
       K22479    17             17      17
       K22486     2              2       2
       K22491     3              3       3
       K22503     5              5       5

[6452 rows x 3 columns]

In [272]:
a = dff.loc[dff["site_y"] == "1.1"]["KEGG_ko"].dropna("KEGG_ko").str.split(",").explode(preserve_index=True).str[3:]
a

TypeError: Series.explode() got an unexpected keyword argument 'preserve_index'

In [157]:
b = {}
for i in df["site"].unique().tolist():
    a = df.loc[df["site"] == i]["KEGG_ko"].dropna().str.split(",").explode().str[3:]
    av = a.value_counts()
    b[i] = av
kegg_df = pd.concat(b, axis=1)
kegg_df = kegg_df.replace(np.nan, 0)
kegg_df

,1,2,3,4,5
KEGG_ko,,,,,
K12132,2288.0,1993.0,963.0,1555.0,1528.0
K03088,2241.0,2142.0,1443.0,2843.0,1784.0
K01989,1469.0,3905.0,1798.0,1294.0,605.0
K00059,1448.0,1409.0,893.0,1283.0,1131.0
K01990,1407.0,1477.0,1154.0,2071.0,1101.0
...,...,...,...,...,...
K15153,0.0,0.0,0.0,0.0,1.0
K20315,0.0,0.0,0.0,0.0,1.0
K12301,0.0,0.0,0.0,0.0,1.0


In [158]:
kegg_df.to_csv("kegg_df.csv")

In [59]:
a = df.loc[df["site"] == "1"]["KEGG_ko"].dropna().str.split(",").explode().str[3:]
av = a.value_counts()

In [149]:
pd.concat({"1":av1, "2":av2}, axis=1)

,1,2
KEGG_ko,,
K12132,2288.0,1993.0
K03088,2241.0,2142.0
K01989,1469.0,3905.0
K00059,1448.0,1409.0
K01990,1407.0,1477.0
...,...,...
K08960,NaN,1.0
K14758,NaN,1.0
K21484,NaN,1.0


In [88]:
a1 = df.loc[df["site"] == "1"]["KEGG_ko"].dropna().str.split(",").explode().str[3:]
av1 = a1.value_counts()
avdf1 = pd.DataFrame(av1)
avdf1.columns = ["hui1"]
avdf1

,hui1
KEGG_ko,
K12132,2288
K03088,2241
K01989,1469
K00059,1448
K01990,1407
...,...
K01727,1
K04373,1
K04345,1


In [130]:
av2

KEGG_ko
K01989    3905
K03088    2142
K12132    1993
K01990    1477
K02035    1450
          ... 
K21483       1
K21484       1
K12537       1
K16794       1
K14761       1
Name: count, Length: 6754, dtype: int64

In [138]:
pd.concat([av1, av2], axis=1)

,count,count
KEGG_ko,,
K12132,2288.0,1993.0
K03088,2241.0,2142.0
K01989,1469.0,3905.0
K00059,1448.0,1409.0
K01990,1407.0,1477.0
...,...,...
K08960,NaN,1.0
K14758,NaN,1.0
K21484,NaN,1.0


In [122]:
nulldf = pd.DataFrame.empty()
nulldf.combine_first(avdf2)

TypeError: 'property' object is not callable

In [106]:
a2 = df.loc[df["site"] == "2"]["KEGG_ko"].dropna().str.split(",").explode().str[3:]
av2 = a2.value_counts()
avdf2 = pd.DataFrame(av2)
avdf2.columns = ["hui2"]
avdf2

,hui2
KEGG_ko,
K01989,3905
K03088,2142
K12132,1993
K01990,1477
K02035,1450
...,...
K21483,1
K21484,1
K12537,1


In [115]:
avdf1.join(avdf2, how="outer")

,hui1,hui2
KEGG_ko,,
K00001,421.0,397.0
K00002,2.0,6.0
K00003,144.0,157.0
K00004,49.0,85.0
K00005,4.0,2.0
...,...,...
K22486,2.0,NaN
K22491,3.0,23.0
K22492,NaN,1.0


In [56]:
pd.DataFrame.from_dict()

""


In [54]:
a.KEGG_ko.map(a.KEGG_ko.value_counts())

AttributeError: 'Series' object has no attribute 'KEGG_ko'

In [26]:
pd.DataFrame.from_dict(b, orient='tight')

KeyError: 'data'

In [20]:
df.groupby["KEGG_ko"].dropna().str.split(",").explode().str[3:]

SyntaxError: invalid syntax (1579050678.py, line 1)

In [169]:
dfpaf = pd.read_csv("~/soil_institute/minimap2/mapping_test.paf",sep="\t", names=["query", "hui1", "hui2", "hui3", "hui4", "target", "hui5", "hui6", "hui7", "hui8", "hui9", "quality", "hui10", "hui11", "hui12", "hui13", "hui14", "hui15"])
dfpaf = dfpaf.drop(list(dfpaf.filter(regex='hui')), axis=1)

## together

In [352]:
files = sorted(glob.glob("/home/gladkovgpu/soil_institute/minimap2/*_depth.txt"))
names = [os.path.basename(i).split("_depth")[0] for i in files]
file_list = list(zip(names, files))
header = pd.read_csv(file_list[1][1],sep="\t",nrows=0).columns.to_list()
dfs = [pd.read_csv(f[1], sep="\t").assign(site=f[0]) for f in file_list]
dfpaf = pd.concat(dfs, ignore_index=True)
dfpaf = dfpaf.drop(list(dfpaf.filter(regex='bam')), axis=1)
dfpaf = dfpaf.drop("contigLen", axis=1)
dfpaf['id'] = dff['site'].str.split(".").str[0]  + "_" + dfpaf['contigName']
dfpaf

,contigName,totalAvgDepth,site,id
0,contig_56336,0.000000,1.1,1_contig_56336
1,contig_29215,0.166206,1.1,1_contig_29215
2,contig_8522,0.121339,1.1,1_contig_8522
3,contig_58210,0.000000,1.1,1_contig_58210
4,contig_17370,0.000000,1.1,1_contig_17370
...,...,...,...,...
443299,contig_914,0.000000,5.3,1_contig_914
443300,contig_2648,0.000000,5.3,1_contig_2648
443301,contig_21596,0.000000,5.3,1_contig_21596
443302,contig_35266,0.000000,5.3,1_contig_35266


In [373]:
dff = df.copy()
dff = dff[["#query", "KEGG_ko", "site"]]
dff['id'] = dff['site'] + "_" + dff['#query'].str.split("_").str[:2].str.join("_") 
dff = dff.merge(dfpaf, on='id', how='left')
dff = dff.dropna(subset=["KEGG_ko", "totalAvgDepth"])
dff["KEGG_ko"] = dff["KEGG_ko"].str.split(",")
dff = dff.explode("KEGG_ko")
dff["KEGG_ko"] = dff["KEGG_ko"].str[3:]
dff["count"] = dff[["id","KEGG_ko","totalAvgDepth", "site_y", "site_x"]].groupby(["site_y","KEGG_ko"]).transform('count')["id"]
dff = dff[["id","KEGG_ko","totalAvgDepth", "site_y", "site_x", "count"]].drop_duplicates(ignore_index=True)
dff["weight"] = dff["count"]*dff["totalAvgDepth"]
dff = dff[["KEGG_ko","site_y", "weight"]].groupby(["KEGG_ko","site_y"]).sum()
dff

weight
KEGG_ko site_y              
K00001  1.1     19652.061417
        1.2     22500.081919
        1.3     29377.718156
        2.1      5513.969439
        2.2      6129.177898
...                      ...
K22503  4.2         0.000000
        4.3         0.000000
        5.1         0.058844
        5.2         0.080155
        5.3         0.087058

[80445 rows x 1 columns]

In [375]:
dff = dff.reset_index()
dff.loc[dff["KEGG_ko"] == "K00001"]

,index,KEGG_ko,site_y,weight
0,0,K00001,1.1,19652.061417
1,1,K00001,1.2,22500.081919
2,2,K00001,1.3,29377.718156
3,3,K00001,2.1,5513.969439
4,4,K00001,2.2,6129.177898
5,5,K00001,2.3,3784.096764
6,6,K00001,3.1,67.971912
7,7,K00001,3.2,121.517312
8,8,K00001,3.3,135.377727
9,9,K00001,4.1,1143.728774


In [376]:
dff.to_csv("kegg_df.csv")

In [360]:
dff.loc[dff["site_y"] == "5.1"]

,#query,KEGG_ko,site_x,id,contigName,totalAvgDepth,site_y
12,contig_1_1,ko:K01601,1,1_contig_1,contig_1,0.005648,5.1
27,contig_1_2,NaN,1,1_contig_1,contig_1,0.005648,5.1
42,contig_1_3,NaN,1,1_contig_1,contig_1,0.005648,5.1
57,contig_1_4,ko:K01783,1,1_contig_1,contig_1,0.005648,5.1
72,contig_1_0,ko:K02347,1,1_contig_1,contig_1,0.005648,5.1
...,...,...,...,...,...,...,...
3755763,contig_9999_3,NaN,1,1_contig_9999,contig_9999,0.000000,5.1
3755772,contig_9999_1,NaN,1,1_contig_9999,contig_9999,0.000000,5.1
3755781,contig_9999_14,NaN,1,1_contig_9999,contig_9999,0.000000,5.1
3755790,contig_9999_5,NaN,1,1_contig_9999,contig_9999,0.000000,5.1


In [181]:
dfpaf.groupby("target").size()

target
contig_1          8
contig_10001      5
contig_10004     31
contig_10005     14
contig_10006    449
               ... 
contig_9992     150
contig_9996      55
contig_9997      31
contig_9998      35
contig_9999     114
Length: 41321, dtype: int64

In [61]:
%ll -h 

total 22M
-rw-rw-r-- 1 gladkovgpu 3.8M Jul 28 14:12 1_keggs
-rw-rw-r-- 1 gladkovgpu 4.2M Jul 28 14:12 2_keggs
-rw-rw-r-- 1 gladkovgpu 3.0M Jul 28 14:12 3_keggs
-rw-rw-r-- 1 gladkovgpu 4.7M Jul 28 14:12 4_keggs
-rw-rw-r-- 1 gladkovgpu 3.5M Jul 28 14:12 5_keggs
-rw-rw-r-- 1 gladkovgpu 487K Jul 28 14:13 detailed_output_minpath_site_1
-rw-rw-r-- 1 gladkovgpu 515K Jul 28 14:13 detailed_output_minpath_site_2
-rw-rw-r-- 1 gladkovgpu 449K Jul 28 14:14 detailed_output_minpath_site_3
-rw-rw-r-- 1 gladkovgpu 488K Jul 28 14:14 detailed_output_minpath_site_4
-rw-rw-r-- 1 gladkovgpu 472K Jul 28 14:15 detailed_output_minpath_site_5
-rw-rw-r-- 1 gladkovgpu  41K Jul 28 14:13 result_minpath_site_1
-rw-rw-r-- 1 gladkovgpu  43K Jul 28 14:13 result_minpath_site_2
-rw-rw-r-- 1 gladkovgpu  39K Jul 28 14:14 result_minpath_site_3
-rw-rw-r-- 1 gladkovgpu  41K Jul 28 14:14 result_minpath_site_4
-rw-rw-r-- 1 gladkovgpu  41K Jul 28 14:15 result_minpath_site_5


## Minpath after limma

In [400]:
!ls ../postprocessing/pod_cher

less_forest_kegg.csv


In [ ]:
!python /home/gladkovgpu/app/MinPath/MinPath.py -ko  ../postprocessing/pod_cher/less_forest_kegg.csv -report result_less_forest -details detailed_output_less_forest

In [397]:
!ls --color

Untitled.ipynb                     clusters.ipynb           results
al.bam                             docu-docu.ipynb          results2
al.bam.csi                         edges                    scripts.bash
assembly.fasta.split               iframe_figures           second_run
assembly_depth.txt                 kegg_df.csv              test
assembly_graph.gfa                 marker_gene_stats.tsv    test.mps
checkm_edges                       mlruns
checkm_edges_polished_results.txt  original_assembly.fasta


In [398]:
!ls /home/gladkovgpu/app/MinPath/data

KEGG-family.txt   KEGG-pathway.txt    MetaCyc-mapping.txt  ec2path
KEGG-mapping.txt  MetaCyc-family.txt  MetaCyc-pathway.txt  figfam_subsystem.dat


In [404]:
from pathlib import Path

path = r'/mnt/storage/soil_institute/postprocessing/pod_cher'
files = Path(path).glob('*_forest')

dfs = list()
for f in files:
    data = pd.read_csv(f,sep="  ",engine='python', names=["hui0", "hui1", "hui2", "hui22", "fam0", "hui3", "fam_found", "hui5", "name"])
    data['site'] = f.stem
    dfs.append(data)

minpath = pd.concat(dfs, ignore_index=True)
minpath = minpath.drop(list(minpath.filter(regex='hui')), axis=1)
cols = minpath.columns.tolist()
cols = cols[-1:] + cols[:-1]
minpath = minpath[cols]
minpath["site"] = minpath["site"].str.split("_").str[1]
minpath = minpath.sort_values("site")
minpath


,site,fam0,fam_found,name
256,less,70,10,"Valine, leucine and isoleucine degradation"
350,less,7,1,Betalain biosynthesis
349,less,26,2,"Tropane, piperidine and pyridine alkaloid bios..."
348,less,62,2,Isoquinoline alkaloid biosynthesis
347,less,15,1,Flavone and flavonol biosynthesis
...,...,...,...,...
148,more,63,4,PPAR signaling pathway
147,more,74,23,Bacterial secretion system
146,more,39,14,Protein export
144,more,51,4,RNA polymerase


In [405]:
minpath["rel"] = minpath["fam_found"].div(minpath.groupby('site')["fam_found"].transform('sum'))
minpath["rel"] = minpath["rel"].transform(lambda x: round(x*100, 2))
min_sorted = minpath.sort_values("fam_found", ascending=False).groupby("site").head(3000)
min_sorted

,site,fam0,fam_found,name,rel
373,less,512,108,ABC transporters,6.60
374,less,495,95,Two-component system,5.80
134,more,495,65,Two-component system,3.74
141,more,143,53,Ribosome,3.05
133,more,512,47,ABC transporters,2.70
...,...,...,...,...,...
306,less,15,1,Glycosphingolipid biosynthesis - globo and iso...,0.06
410,less,103,1,Meiosis - yeast,0.06
371,less,77,1,Endocrine resistance,0.06
332,less,13,1,Lipoic acid metabolism,0.06


In [406]:
import plotly.express as px


fig = px.bar(min_sorted, x="site", y="rel", color="name", title="Minpath output")
fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['1','2','3','4', '5']})
fig.update_layout(autosizeFalse,width=1200,height=800)
fig.show()

In [ ]:
%%bash

for file in /mnt/storage/soil_institute/postprocessing/pod_cher/*; do
    fname=$(basename $file)
    echo $fname
    python /home/gladkovgpu/app/MinPath/MinPath.py -ko $file -report vroommin_${fname}
done

In [417]:
!pwd

/mnt/storage/soil_institute/graphmb


In [487]:
from pathlib import Path

path = r'/mnt/storage/soil_institute/graphmb/'
files = Path(path).glob('vroom*')

dfs = list()
for f in files:
    data = pd.read_csv(f,sep="  ",engine='python', names=["hui0", "hui1", "hui2", "hui22", "fam0", "hui3", "fam_found", "hui5", "name"])
    data['site'] = f.stem
    dfs.append(data)

minpath = pd.concat(dfs, ignore_index=True)
minpath = minpath.drop(list(minpath.filter(regex='hui')), axis=1)
cols = minpath.columns.tolist()
cols = cols[-1:] + cols[:-1]
minpath = minpath[cols]
minpath["lfc"] = minpath["site"].str.split("_").str[2]
minpath["site"] = minpath["site"].str.split("_").str[1]
minpath = minpath.sort_values("site")
minpath


,site,fam0,fam_found,name,lfc
0,1,105,9,Glycolysis / Gluconeogenesis,less
316,1,12,1,Arabinogalactan biosynthesis - Mycobacterium,more
315,1,113,9,Glycerophospholipid metabolism,more
314,1,76,4,Inositol phosphate metabolism,more
313,1,90,6,Glycerolipid metabolism,more
...,...,...,...,...,...
1894,5,131,2,Tuberculosis,more
1893,5,74,1,Staphylococcus aureus infection,more
1892,5,87,1,Amoebiasis,more
1902,5,38,1,Bladder cancer,more


In [488]:
minpath["rel"] = minpath["fam_found"].div(minpath.groupby('site')["fam_found"].transform('sum'))
minpath["rel"] = minpath["rel"].transform(lambda x: round(x*100, 2))
min_sorted = minpath.sort_values("fam_found", ascending=False).groupby("site").head(3000)
min_sorted

,site,fam0,fam_found,name,lfc,rel
788,3,512,130,ABC transporters,more,3.74
789,3,495,107,Two-component system,more,3.08
131,1,512,84,ABC transporters,less,3.37
132,1,495,66,Two-component system,less,2.64
2039,2,512,57,ABC transporters,less,2.59
...,...,...,...,...,...,...
209,1,14,1,Proximal tubule bicarbonate reclamation,less,0.04
210,1,50,1,Gastric acid secretion,less,0.04
211,1,66,1,Protein digestion and absorption,less,0.04
213,1,36,1,Mineral absorption,less,0.04


In [489]:
min_sorted['rel'] = np.where( min_sorted.lfc == "more", 0 - min_sorted['rel'], min_sorted['rel'])

In [490]:
min_sorted

,site,fam0,fam_found,name,lfc,rel
788,3,512,130,ABC transporters,more,-3.74
789,3,495,107,Two-component system,more,-3.08
131,1,512,84,ABC transporters,less,3.37
132,1,495,66,Two-component system,less,2.64
2039,2,512,57,ABC transporters,less,2.59
...,...,...,...,...,...,...
209,1,14,1,Proximal tubule bicarbonate reclamation,less,0.04
210,1,50,1,Gastric acid secretion,less,0.04
211,1,66,1,Protein digestion and absorption,less,0.04
213,1,36,1,Mineral absorption,less,0.04


In [481]:
minpath["rel"] = minpath["fam_found"].div(minpath.groupby('site')["fam_found"].transform('sum'))
minpath["rel"] = minpath["rel"].transform(lambda x: round(x*100, 2))

min_sorted = minpath.sort_values("fam_found", ascending=False).groupby("site").head(3000)
min_sorted

,site,fam0,fam_found,name,lfc,rel
788,3,512,130,ABC transporters,more,3.74
789,3,495,107,Two-component system,more,3.08
131,1,512,84,ABC transporters,less,3.37
132,1,495,66,Two-component system,less,2.64
2039,2,512,57,ABC transporters,less,2.59
...,...,...,...,...,...,...
209,1,14,1,Proximal tubule bicarbonate reclamation,less,0.04
210,1,50,1,Gastric acid secretion,less,0.04
211,1,66,1,Protein digestion and absorption,less,0.04
213,1,36,1,Mineral absorption,less,0.04


In [491]:
import plotly.express as px

fig = px.bar(min_sorted, x="site", y="rel", color="name", title="Minpath output")
fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['1','2','3','4', '5']})
fig.update_layout(autosize = False,width=1200,height=900)
fig.show()

In [469]:
dfff = dff.copy()
# df.loc[df["site"] == i]["KEGG_ko"].dropna().str.split(",")

In [470]:
dfff["site_y"] = dfff["site_y"].str.split(".").str[0]

In [471]:
dfff = dfff[["KEGG_ko","site_y", "weight"]].groupby(["KEGG_ko","site_y"]).sum()
dfff = dfff.reset_index()

In [472]:
dfff

,KEGG_ko,site_y,weight
0,K00001,1,71529.861492
1,K00001,2,15427.244101
2,K00001,3,324.866951
3,K00001,4,5622.498076
4,K00001,5,11113.830306
...,...,...,...
26810,K22491,5,0.149107
26811,K22503,1,10.139985
26812,K22503,2,2.756154
26813,K22503,4,0.000000


In [474]:
dfff["rel"] = dfff["weight"].div(dfff.groupby('site_y')["weight"].transform('sum'))
dfff["rel"] = dfff["rel"].transform(lambda x: round(x*100, 2))
dfff = dfff.sort_values("rel", ascending=False).groupby("site_y").head(3000)
dfff

,KEGG_ko,site_y,weight,rel
5528,K01989,3,3.422273e+04,20.25
8005,K03088,2,3.977539e+05,8.81
5529,K01989,4,1.382235e+05,8.43
4827,K01768,3,1.299757e+04,7.69
18575,K12132,1,1.872888e+06,7.13
...,...,...,...,...
14575,K07649,3,0.000000e+00,0.00
14593,K07654,3,0.000000e+00,0.00
14613,K07659,3,1.203011e+00,0.00
14608,K07658,3,4.832528e-01,0.00


In [495]:
import plotly.express as px

fig = px.bar(dfff, x="site_y", y="rel", color="KEGG_ko", title="KEGG categories relative abundance")
fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['1','2','3','4', '5']})
fig.update_layout(autosize=False,width=1200,height=800)
fig.show()